In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import adfuller
from sklearn.linear_model import BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import TimeSeriesSplit


In [2]:
data = pd.read_csv("../data/HMIS_DATA_CORRECTED_17_21/mh_dist17_21_with_IDs_date_correction.csv")
data = data[(data['indicator_type'] == 'Total [(A+B) or (C+D)]')]
data['date'] = pd.to_datetime(data['date'])
data = data.set_index('date')
data.index = pd.DatetimeIndex(data.index)

In [3]:
def bayesian_ridge_regression_lags_only(
    series,
    district_name,
    max_lags=3,
    differencing=True,
    n_iter=300,
    alpha_1=1e-6,
    alpha_2=1e-6,
    lambda_1=1e-6,
    lambda_2=1e-6,
    compute_scores=True,
    random_state=None
):
    """
    Bayesian Ridge Regression for time series forecasting with:
    - Probabilistic uncertainty estimates
    - Automatic complexity control
    - Lag features only
    """
    # Create directory structure
    os.makedirs('BayesianRidgeRegression', exist_ok=True)
    
    # 1. Stationarity and Differencing
    original_series = series.copy()
    d = 0
    if differencing:
        adf_result = adfuller(series.dropna())
        if adf_result[1] > 0.05:
            d = 1
            series = series.diff().dropna()

    # 2. Feature Engineering - Lag features
    df = pd.DataFrame({'y': series.astype(float)})
    for lag in range(1, max_lags + 1):
        df[f'lag_{lag}'] = df['y'].shift(lag).astype(float)
    df = df.dropna()

    # 3. Train-Test Split
    train_size = int(len(df) * 0.8)
    train = df.iloc[:train_size]
    test = df.iloc[train_size:]

    X_train = train.drop(columns=['y'])
    y_train = train['y']
    X_test = test.drop(columns=['y'])
    y_test = test['y']

    # 4. Model Training with Bayesian Ridge
    model = BayesianRidge(
        n_iter=n_iter,
        alpha_1=alpha_1,
        alpha_2=alpha_2,
        lambda_1=lambda_1,
        lambda_2=lambda_2,
        compute_score=compute_scores,
    )
    model.fit(X_train, y_train)

    # 5. Forecasting with Uncertainty
    pred_test, pred_test_std = model.predict(X_test, return_std=True)

    # 6. Inverse Differencing
    if d == 1:
        last_train_value = original_series.iloc[len(original_series) - len(test) - 1]
        pred_test = np.cumsum(pred_test) + last_train_value
        y_test = original_series.iloc[-len(test):]

    # 7. Create forecast DataFrame with uncertainty
    forecast_df = pd.DataFrame({
        'district': district_name,
        'date': y_test.index,
        'actual': y_test.values,
        'forecast_mean': pred_test,
        'forecast_std': pred_test_std
    })

    # 8. Save forecasts
    forecast_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_forecasts.csv')
    write_header = not os.path.exists(forecast_csv_path)
    forecast_df.to_csv(forecast_csv_path, mode='a', header=write_header, index=False)

    # 9. Calculate metrics and uncertainty statistics
    rmse = np.sqrt(mean_squared_error(forecast_df['actual'], forecast_df['forecast_mean']))
    log_marginal_likelihood = model.scores_[-1] if compute_scores else np.nan
    
    # Calculate coefficient statistics
    coef_means = model.coef_
    coef_stds = np.sqrt(np.diag(model.sigma_))
    
    metrics_df = pd.DataFrame([{
        'district': district_name,
        'rmse': rmse,
        'log_marginal_likelihood': log_marginal_likelihood,
        'coefficient_means': coef_means.tolist(),
        'coefficient_stds': coef_stds.tolist(),
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan,
        'effective_params': model.lambda_ / model.alpha_,
        'differencing': d,
        'n_iter': n_iter
    }])

    metrics_csv_path = os.path.join('BayesianRidgeRegression', 'bayesian_metrics.csv')
    write_header_metrics = not os.path.exists(metrics_csv_path)
    metrics_df.to_csv(metrics_csv_path, mode='a', header=write_header_metrics, index=False)

    # 10. Plotting with uncertainty bands
    plt.figure(figsize=(12, 6))
    plt.plot(original_series.index, original_series.values, label='Original Series')
    plt.plot(forecast_df['date'], forecast_df['forecast_mean'], label='Mean Forecast', color='darkorange')
    plt.fill_between(
        forecast_df['date'],
        forecast_df['forecast_mean'] - 1.96*forecast_df['forecast_std'],
        forecast_df['forecast_mean'] + 1.96*forecast_df['forecast_std'],
        color='orange',
        alpha=0.2,
        label='95% Confidence'
    )
    plt.title(
        f'Bayesian Ridge Forecast for {district_name}\n'
        f'RMSE: {rmse:.2f} | Log ML: {log_marginal_likelihood:.2f}'
    )
    plt.legend()
    plt.savefig(os.path.join('BayesianRidgeRegression', f'bayesian_forecast_{district_name}.png'))
    plt.close()

    return {
        'district': district_name,
        'forecast_df': forecast_df,
        'metrics_df': metrics_df,
        'model': model,
        'coef_means': coef_means,
        'coef_stds': coef_stds,
        'intercept_mean': model.intercept_,
        'intercept_std': np.nan
    }

In [4]:
districts = data['district'].unique()
rmse_values = []

def run_for_each_district():
    results = {}
    
    for district in districts:
        district_data = data[data['district'] == district]
        ts = district_data["I40"].asfreq('MS')
        
        results = bayesian_ridge_regression_lags_only(
            ts,
            district,
            max_lags=4,
            n_iter=500,
            alpha_1=1e-5,
            lambda_1=1e-3
        )
        
    
        # Show results
        print("=== Metrics ===")
        print(results['metrics_df'])
        print("\n=== Forecast Data ===")
        print(results['forecast_df'].head())
    
    return results
run_for_each_district()

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
     district       rmse  log_marginal_likelihood  \
0  AHMEDNAGAR  15.812603              -133.291919   

                                   coefficient_means  \
0  [0.0034438916536789622, -0.0001901844947154823...   

                                    coefficient_stds  intercept_mean  \
0  [0.018740734927776857, 0.018737301824296237, 0...       43.700926   

   intercept_std  effective_params  differencing  n_iter  
0            NaN     333996.938669             0     500  

=== Forecast Data ===
              district       date  actual  forecast_mean  forecast_std
date                                                                  
2020-07-01  AHMEDNAGAR 2020-07-01    40.0      43.936148     10.987037
2020-08-01  AHMEDNAGAR 2020-08-01    55.0      43.933439     10.987042
2020-09-01  AHMEDNAGAR 2020-09-01    36.0      43.993783     11.017748
2020-10-01  AHMEDNAGAR 2020-10-01    82.0      43.931347     11.017671
2020-11-01  AHMEDNAGAR 2020-11-01    49.0      44.11

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
  district       rmse  log_marginal_likelihood  \
0    AKOLA  11.008634              -132.707331   

                                   coefficient_means  \
0  [0.14113531980791238, 0.13091907469375633, 0.0...   

                                    coefficient_stds  intercept_mean  \
0  [0.1121157745601413, 0.11301857421922183, 0.11...       33.628315   

   intercept_std  effective_params  differencing  n_iter  
0            NaN       4861.023467             0     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01    AKOLA 2020-07-01    34.0      45.406104     12.679692
2020-08-01    AKOLA 2020-08-01    45.0      44.619256     12.687100
2020-09-01    AKOLA 2020-09-01    54.0      45.696574     12.781596
2020-10-01    AKOLA 2020-10-01    48.0      48.289136     13.205992
2020-11-01    AKOLA 2020-11-01    39.0      48.656983     13.443394


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


...

=== Metrics ===
  district     rmse  log_marginal_likelihood  \
0   WASHIM  4.20541               -95.022618   

                                   coefficient_means  \
0  [0.09002989152908955, -0.021455599763839917, -...   

                                    coefficient_stds  intercept_mean  \
0  [0.0898317634579432, 0.09200730096592073, 0.09...        8.789563   

   intercept_std  effective_params  differencing  n_iter  
0            NaN       1145.134649             0     500  

=== Forecast Data ===
           district       date  actual  forecast_mean  forecast_std
date                                                               
2020-07-01   WASHIM 2020-07-01    13.0       9.033511      4.044284
2020-08-01   WASHIM 2020-08-01    18.0       8.901882      4.172299
2020-09-01   WASHIM 2020-09-01     7.0       9.053828      4.452133
2020-10-01   WASHIM 2020-10-01    10.0       8.120569      4.216252
2020-11-01   WASHIM 2020-11-01     5.0       8.363911      4.214847


c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


=== Metrics ===
   district      rmse  log_marginal_likelihood  \
0  YAVATMAL  6.683218              -114.460736   

                                   coefficient_means  \
0  [0.41494245675227354, 0.22008702384187856, -0....   

                                    coefficient_stds  intercept_mean  \
0  [0.14706047505338046, 0.15784777884710585, 0.1...        4.836494   

   intercept_std  effective_params  differencing  n_iter  
0            NaN          439.1632             0     500  

=== Forecast Data ===
            district       date  actual  forecast_mean  forecast_std
date                                                                
2020-07-01  YAVATMAL 2020-07-01     3.0       6.308960      6.372975
2020-08-01  YAVATMAL 2020-08-01     7.0       6.023115      5.981038
2020-09-01  YAVATMAL 2020-09-01     8.0       8.382488      5.997124
2020-10-01  YAVATMAL 2020-10-01    23.0       9.543263      5.995167
2020-11-01  YAVATMAL 2020-11-01     6.0      15.786776      6.582617
=

c:\Users\nauti\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\linear_model\_bayes.py:54: FutureWarning: 'n_iter' was renamed to 'max_iter' in version 1.3 and will be removed in 1.5
  warnings.warn(


{'district': 'MUMBAI SUBURBAN',
 'forecast_df':                    district       date  actual  forecast_mean  forecast_std
 date                                                                       
 2021-02-01  MUMBAI SUBURBAN 2021-02-01      28      29.585505      9.751985
 2021-03-01  MUMBAI SUBURBAN 2021-03-01      28      35.514140      8.665135,
 'metrics_df':           district      rmse  log_marginal_likelihood  \
 0  MUMBAI SUBURBAN  5.430291               -19.325982   
 
                                    coefficient_means  \
 0  [-0.6332501940717783, 0.20732265664917227, -0....   
 
                                     coefficient_stds  intercept_mean  \
 0  [0.3144703711635698, 0.3879400634302669, 0.431...       -5.889937   
 
    intercept_std  effective_params  differencing  n_iter  
 0            NaN        156.642563             1     500  ,
 'model': BayesianRidge(alpha_1=1e-05, compute_score=True, lambda_1=0.001, n_iter=500),
 'coef_means': array([-0.63325019,  0.2